In [160]:
import pandas as pd
import zipfile
import os
import requests
import io
from Levenshtein import distance
import time
import numpy as np

In [161]:
#Establecer directorio de trabajo
os.chdir("D:/github/correcci-n_nombres")

In [162]:
#Identificar archivos csv en la carpeta raw_data
archivos = os.listdir("raw_data")
archivos_csv = [i for i in archivos if i.endswith(".csv")]

#Leer y pegar los archivos csv usando un loop
df = pd.DataFrame()
for i in archivos_csv:
    df = pd.concat([df, pd.read_csv("raw_data/" + i, encoding = "utf-8", low_memory = False)],
                   axis = 0)
#Columnas en minúsculas
df.columns = df.columns.str.lower()

#Seleccionar columnas 3,4 y 5
df = df.iloc[:,[3,4,5]]

#Renombrar columnas
df.columns = ["primer_apellido", "segundo_apellido", "nombre"]

#Eliminar los registros "MENOR" en las tres columnas usando un loop
for i in df.columns:
    df = df[df[i] != "MENOR"]
    #Eliminar nans de todas las columnas
    df = df.dropna()


In [163]:
df.shape

(7905934, 3)

In [164]:
#Obtener el número máximo de espacios que tiene la columna nombre
max(df.nombre.str.count(" "))

11

In [165]:
#obtener el nombre con el número máximo de espacios
#df[df.nombre.str.count(" ") == 11]

In [166]:
#Buscar registros con apellido=="AAGUILAR"
#df[df.primer_apellido == "AAGUILAR"]

In [167]:
#Dejar únicamente el primer nombre
df["nombre"] = df["nombre"].str.split(" ").str[0]

In [168]:
#Contar si existen nans en las columnas
df.isna().sum()

primer_apellido     0
segundo_apellido    0
nombre              0
dtype: int64

In [169]:
#Crear una lista de apellidos y nombres
apellidos = df.primer_apellido.unique().tolist() + df.segundo_apellido.unique().tolist()
nombres = df.nombre.unique().tolist()
#Hacer una lista general de nombres y apellidos
nombres_apellidos = apellidos + nombres
#Ordenar listas
apellidos.sort()
nombres.sort()
nombres_apellidos.sort()

In [170]:
#Obtener el número de elementos en la lista de nombres
print("Total de nombres: ",len(nombres))
print("Total de apellidos: ",len(apellidos))
print("Total nombres+apellidos: ",len(nombres_apellidos))

Total de nombres:  128033
Total de apellidos:  89710
Total nombres+apellidos:  217743


In [171]:
#Cargar archivo de excel con nombres a corregir
nombres_corregir = pd.read_excel("Nombres_SEMEFO_Anotaciones.xlsx")
#Dejar únicamente la variable FourthConfig
nombres_corregir = nombres_corregir.iloc[:,[4]]
#Eliminar NaNs
nombres_corregir = nombres_corregir.dropna()
#Separar FourthConfig en el mayor número de columnas posibles
nombres_corregir = nombres_corregir["FourthConfig"].str.split(" ", expand = True)
#Dejar solo las primeras 3 columnas
nombres_corregir = nombres_corregir.iloc[:,[0,1,2]]
#Renombrar columnas
nombres_corregir.columns = ["primer_apellido", "segundo_apellido", "nombre"]

In [172]:
nombres_corregir

,primer_apellido,segundo_apellido,nombre
0,CHTAUTLLO,None,None
1,ENON_,BADILLO,GARCIA
2,BRO,Q,LE
3,EZEQUEL,None,None
4,COCCION,SEG,TITO
...,...,...,...
256,DEPOSIJ,None,None
257,ANTONIO,CASTILLO,ESPINO
258,JOSE,CALDERON,MORALES
259,CIA,None,None


In [173]:
start_time = time.perf_counter()

# Iterar sobre cada fila de la columna nombre_mal
for i in nombres_corregir["primer_apellido"]:
    #Obtener la distancia de Levenshtein entre el nombre a corregir y cada uno de los nombres de la lista
    distancias = [distance(i, j) for j in nombres]
    #Obtener el índice de los 3 nombres más cercanos
    indices = np.argsort(distancias)[:3]
    #Obtener los 3 nombres más cercanos
    nombres_mas_cercanos = [nombres[i] for i in indices]
    #Crear 3 columnas con los 3 nombres más cercanos
    nombres_corregir.loc[nombres_corregir["primer_apellido"] == i, "nombre_cercano_1"] = nombres_mas_cercanos[0]
    #nombres_corregir.loc[nombres_corregir["primer_apellido"] == i, "nombre_cercano_2"] = nombres_mas_cercanos[1]
    #nombres_corregir.loc[nombres_corregir["primer_apellido"] == i, "nombre_cercano_3"] = nombres_mas_cercanos[2]
    #Ordenar por primer_apellido
    nombres_corregir = nombres_corregir.sort_values(by = "primer_apellido")

end_time = time.perf_counter()

#Imprimir el tiempo de ejecución
print("Número de elementos revisados: ", len(nombres_corregir))
print(f'Tiempo de ejecución: {end_time - start_time} segundos')
print("Tiempo por elemento: ", (end_time - start_time)/len(nombres_corregir), " segundos")

nombres_corregir

Número de elementos revisados:  257
Tiempo de ejecución: 50.89545559999897 segundos
Tiempo por elemento:  0.19803679221789483  segundos


,primer_apellido,segundo_apellido,nombre,nombre_cercano_1
152,2,PABLO,MARTINEZ,
198,2,E,RAMA,
85,28404819983,0,None,
52,A,A,None,A
217,A,SCULINO,DE,A
...,...,...,...,...
191,___,ALFONSO,ROM,
149,_____,JOEL,ANGEL,
200,_________,PLACTDO,JERONTMO,
58,_________,JOSE,VILLA,
